In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df = pd.read_csv('/Users/vidishapurohit/Desktop/v/Study/Semester6/pre_ana./sampling_ass/Creditcard_data.csv')

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [5]:
df['Class'].value_counts()

0    763
1      9
Name: Class, dtype: int64

In [32]:
# oversampling
from imblearn.over_sampling import SMOTE 

df1 = df.copy()

print('Original class:')
print(df1['Class'].value_counts())

xf = df1.columns
x = df1.drop(['Class'],axis=1)
y = df1['Class']

smote = SMOTE()
x,y = smote.fit_resample(x,y)

df1 = pd.DataFrame(x, columns=xf)
df1['Class'] = y

print('\nNew class after applying SMOTE Technique:')
print(y.value_counts())

Original class:
0    763
1      9
Name: Class, dtype: int64

New class after applying SMOTE Technique:
0    763
1    763
Name: Class, dtype: int64


In [81]:
df1.head()
population= len(df1)

In [83]:
# calculating sample size
import math
from scipy.stats import norm

def sample_size(Z,E,p):
    z_score = norm.ppf((Z + 1) / 2)
    sample_size = (z_score**2 * p * (1-p)) / E**2

    sample_size = math.ceil(sample_size)
    return sample_size

n = sample_size(0.95,0.05,0.5)
n

385

In [166]:
# 5 samples

# simple random
s1 = df1.sample(n, random_state=42)


# systamatic random
sample_interval = population // n
s2 = df1.iloc[::sample_interval]


# stratified random 
from sklearn.model_selection import train_test_split

stratify_column = 'Class'
s3, _ = train_test_split(df1, test_size=(1 - n / len(df1)), stratify=df1[stratify_column], random_state=42)


#cluster
df1['Cluster'] = np.random.choice([1, 2, 3], size=len(df1))
num_clusters_to_select = int(np.ceil(n / 3))
selected_clusters = np.random.choice(df1['Cluster'].unique(), size=num_clusters_to_select, replace=True)
s4 = df1[df1['Cluster'].isin(selected_clusters)].copy()
s4 = s4.sample(n=n, replace=True, random_state=42)


#k fold
from sklearn.model_selection import KFold

X = df1.drop("Class",axis=1)
Y = df1["Class"]

num_folds = 4 

kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
s5 = pd.DataFrame()


# Perform K-fold sampling
fold_number = 1
for train_index, test_index in kf.split(X):
    test_data = X.iloc[test_index]
    test_labels = Y.iloc[test_index]
    
    s5 = pd.concat([s5, pd.concat([test_data, test_labels], axis=1)])
    
    fold_number += 1
    if len(s5) >= n:
        break

s5 = s5.head(n)


      Time        V1        V2        V3        V4        V5        V6  \
15      12 -0.752417  0.345485  2.057323 -1.468643 -1.158394 -0.077850   
18      14 -5.401258 -5.450148  1.186305  1.736239  3.049106 -1.763406   
23      18  0.247491  0.277666  1.185471 -0.092603 -1.314394 -0.150116   
29      23  1.059387 -0.175319  1.266130  1.186110 -0.786002  0.578435   
30      24  1.237429  0.061043  0.380526  0.761564 -0.359771 -0.494084   
...    ...       ...       ...       ...       ...       ...       ...   
1517   557 -0.153712  0.217769  0.721517  0.055986  0.244416 -0.702085   
1520   433 -2.611517 -0.138992 -0.505564  3.298389  0.248019 -1.278510   
2        1 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499   
3        1 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203   
5        2 -0.425966  0.960523  1.141109 -0.168252  0.420987 -0.029728   

            V7        V8        V9  ...       V22       V23       V24  \
15   -0.608581  0.003603 -0.436167  ..

In [194]:
#table
Results = pd.DataFrame(np.zeros((5,5)), columns=['S1','S2','S3','S4','S5'])
Results.index=['M1','M2','M3','M4','M5']
Results

,S1,S2,S3,S4,S5
M1,0.0,0.0,0.0,0.0,0.0
M2,0.0,0.0,0.0,0.0,0.0
M3,0.0,0.0,0.0,0.0,0.0
M4,0.0,0.0,0.0,0.0,0.0
M5,0.0,0.0,0.0,0.0,0.0


In [196]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

classifiers = [LogisticRegression, KNeighborsClassifier, RandomForestClassifier,GaussianNB]


for i, classifier in enumerate(classifiers):
    for j, sample in enumerate([s1, s2, s3, s4, s5]):
        train_data, test_data = train_test_split(sample, test_size=0.2, random_state=42)
        
        # Ensure the target variable is binary
        y_train = (train_data["Class"] > 0).astype(int)
        y_test = (test_data["Class"] > 0).astype(int)
        
        model = classifier()
        model.fit(train_data.drop("Class", axis=1), y_train)
        y_pred = model.predict(test_data.drop("Class", axis=1))

        accuracy = accuracy_score(y_test, y_pred)
        Results.iloc[i, j] = accuracy

# Display the results
print(Results)


/Users/vidishapurohit/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/vidishapurohit/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

          S1        S2        S3        S4        S5
M1  0.883117  0.852941  0.909091  0.948052  0.870130
M2  0.714286  0.784314  0.675325  0.896104  0.701299
M3  0.961039  0.990196  0.974026  1.000000  0.987013
M4  0.727273  0.872549  0.818182  0.844156  0.753247
M5  0.000000  0.000000  0.000000  0.000000  0.000000
